<a href="https://colab.research.google.com/github/LongaBonga/ChatBot-RUGPT-3/blob/main/%22ChatBot_v.2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Mon Apr 12 12:33:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip3 install -U gitpython
!pip3 install transformers
!pip3 install urllib3
!pip3 install gputil
!pip3 install tabulate

     |████████████████████████████████| 163kB 17.7MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 2.2MB 20.8MB/s 
     |████████████████████████████████| 3.3MB 52.4MB/s 
     |████████████████████████████████| 870kB 54.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=7b86cb5c4c70e9afa82a4ecddea4aad1fd27f525bd3737f152fc0b8aff06d26c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=77ae21a0c471b7a00ae795ed7dfa99d00d3f99a4ac74af77679bfa8944cdec6b
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import pandas as pd
import re
import os
data = pd.read_csv('drive/MyDrive/dialogues.tsv', sep='\t')
data = data.drop(9635)
data = data.reset_index(drop=True)

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import cv2
cv2.setNumThreads(0)

import argparse
from google.colab import drive



if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print('GPU')
else:
  device = torch.device("cpu")

GPU


In [ ]:

import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

model = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2').to(device)
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')



/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
SPECIAL_TOKENS = {'bos_token' : "<bos>", "eos_token" :"<eos>", 'additional_special_tokens': ["<speaker1>", "<speaker2>", "<fos>"]}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
def merge_replyes(buffer):
    history = []
    i = 0
    a = 0
    b = a
    while (i < len(buffer)):
        same = ''
        a = i
        b = a
        while (i < len(buffer) and b < len(buffer) and buffer[a][0] == buffer[b][0]):
            b += 1
        
        for k in range(a, b):
            same += buffer[k][1] + '. '
        history.append([buffer[a][0], same])
        i = b
    return history

In [ ]:
import re
import random
def generate_example(row):
    parsed = re.findall(r'Пользователь [12].+?(?=</span>)', data['dialogue'][row].replace("<br />", " "))
    history = merge_replyes([[1 if sentenses[13] == '1' else 2, sentenses[16:]] for sentenses in parsed])
    # print(history)
    random_slice = random.randint(1, min(len(history) - 1, 18))   # MAKE MAX 20 SENTENSES
    
    if history[random_slice][0] == 1:
        persona = re.findall(r'.+?(?=</span>)', data['persona_2_profile'][row].replace("<br />", " "))[0][26:]
    else:
        persona = re.findall(r'.+?(?=</span>)', data['persona_1_profile'][row].replace("<br />", " "))[0][26:]
        
    reply = history[random_slice][1]
    history = [[i[1]] for i in history[:random_slice]]
    
    
    return history, [reply], persona.split()
    

In [ ]:
generate_example(100)

In [ ]:
from itertools import chain

persona = [["Я", "люблю", "играть", "в", "баскетбол", "."],
           ["Я", "студент", "из", "Москвы", "."]]

history = [["привет", "как", "Дела", "?"],
           ["Все", "хорошо", "спасибо", "."]]

reply = ["Рад", "это", "слышать"]

bos, eos, speaker1, speaker2, fos = "<bos>", "<eos>", "<speaker1>", "<speaker2>", "<fos>"

def build_inputs(persona, history, reply):
    # Build our sequence by adding delimiters and concatenating
    sequence = [[bos] + list(chain(*persona))] + history +  [[fos] + reply + [eos]]
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens
    return words, segments, position, sequence

words, segments, position, sequence = build_inputs(persona, history, reply)
print(words)
print(segments)

In [ ]:
def get_example(index):
  history , reply, persona = generate_example(index)
  words, segments, position, sequence = build_inputs([persona], history, reply)
  # print(words)
  words = tokenizer(words)['input_ids']
  segments = tokenizer(segments)['input_ids']

  not_ansewer_tokens_len = len(list(chain(*words[:-len(sequence[-1])])))
  answer_tokens = list(chain(*words[-len(sequence[-1][1:]):]))
  words = list(chain(*words))
 
  # Prepare our language modeling targets: keep only the reply segment, -1 on the rest
  lm_targets = ([-100] * not_ansewer_tokens_len \
              + [-100] + [-100] + answer_tokens[1:])


  # Store the position of the last tokens for the next-sentence prediction loss
  last_token = len(words) - 1

  input_ids = torch.tensor(words, dtype=torch.long)

  token_type_ids = torch.tensor(segments, dtype=torch.long)


  mc_token_ids = torch.tensor(last_token, dtype=torch.long)


  lm_labels = torch.tensor(lm_targets, dtype=torch.long)
  # Next-sentence prediction labels
  mc_labels = torch.tensor([0], dtype=torch.long)  

  return input_ids, lm_labels

In [ ]:
get_example(20)

(tensor([50258,  1358,  1431,   283,    18, 10150, 12195, 15862,  6890,  8291,
           354,    18,  1140, 24888,  1719,   283,   266,   979, 33784,   297,
            18,  3693,   724,  1719,  2592, 12270,    18,  5218,   623,   267,
          1076,  2844,    18, 50261, 37954,  3715,    23,    18,   225, 50260,
          1295,  8398,     5,   985,  1304,  9140,    16,  4723,   387,    35,
            18,   225, 50261, 26174,  9140, 23646,    16,   367,  1304,    35,
           629,   281, 30407,  2001, 42983,  2356,   269,   376, 49791,    16,
          1314, 29393,   393,   516, 25710,   379,  4934,  4337,  1427,    18,
          1012,   481, 48107, 11331,   889, 24636,    16,   284,  2078,    30,
           816,    18,   439,   387,  4723,    35,    18,   225, 50260, 50262,
         26174,  9140,  6816,    16,   417,   411, 36388,    18, 42148,  1427,
          2002,  5065,   322,  1196,  5656,    18,  1397,  3730, 23554,   309,
         27766,  2240, 11331,   629,    16,  4818,  

In [ ]:
def prepare_input(persona, history):
    sequence = [[bos] + list(chain(*persona))] + history
    sequence = [sequence[0]] + [ [speaker2 if (len(sequence)-i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))                          # word tokens
    segments = [speaker2 if i % 2 else speaker1             # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))                      # position tokens            # position tokens

    print(words)
    words = tokenizer(words)['input_ids']
    words = list(chain(*words))
    input_ids = torch.tensor([words], dtype=torch.long)
    return input_ids

In [ ]:
class Dialoges(Dataset):

    def __init__(self, csv_file):
        data = pd.read_csv(csv_file, sep='\t')
        data = data.drop(9635)
        data = data.reset_index(drop=True)
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return get_example(idx)

In [ ]:
dataset = Dialoges('drive/MyDrive/dialogues.tsv')

train_loader = DataLoader(dataset,
                          batch_size=1, 
                          shuffle=False)

In [ ]:
num_epochs = 3
learning_rate = 0.0000001
warmup_steps = 50
total_steps = len(train_loader) * num_epochs
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
import GPUtil
from tabulate import tabulate

def train(model, loader, optimizer, scheduler, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train', position=0, leave=True)

    model.train()

    cnt = 0
    for X, y in loader:
        X = X.to(device)
        y = y.to(device)
        
        outputs = model(input_ids = X, labels = y)
        loss = outputs[0]
        losses.append(loss.item())

        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()
       

        if cnt % 1000 == 0 and cnt > 1000:
          save_last_model_path = 'drive/MyDrive/last_model_state_dict.pth'
          save_last_optimizer_path = 'drive/MyDrive/last_optimizer_state_dict.pth'
          torch.save(model.state_dict(), save_last_model_path)
          torch.save(optimizer.state_dict(), save_last_optimizer_path)


        if (cnt % 1000 == 0):
          # GPU information

          print("="*40, "GPU Details", "="*40)
          gpus = GPUtil.getGPUs()
          list_gpus = []
          for gpu in gpus:
              # get the GPU id
              gpu_id = gpu.id
              # name of GPU
              gpu_name = gpu.name
              # get % percentage of GPU usage of that GPU
              gpu_load = f"{gpu.load*100}%"
              # get free memory in MB format
              gpu_free_memory = f"{gpu.memoryFree}MB"
              # get used memory
              gpu_used_memory = f"{gpu.memoryUsed}MB"
              # get total memory
              gpu_total_memory = f"{gpu.memoryTotal}MB"
              # get GPU temperature in Celsius
              gpu_temperature = f"{gpu.temperature} °C"
              gpu_uuid = gpu.uuid
              list_gpus.append((
                  gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
                  gpu_total_memory, gpu_temperature, gpu_uuid
              ))

          print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))
        cnt += 1

    progress_bar.close()
    
    return losses

In [ ]:
model.load_state_dict(torch.load('drive/MyDrive/last_model_state_dict.pth'))
optimizer.load_state_dict(torch.load('drive/MyDrive/last_optimizer_state_dict.pth'))

In [ ]:
train_losses = []
train_perplexities = []

for n_epoch in range(1, num_epochs + 1):
    
    epoch_train_losses = train(model, train_loader, optimizer, scheduler)
    mean_train_loss = np.mean(epoch_train_losses)
    
    train_losses.extend(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'  
    print(message) 
    # 1160/10012

Train:   0%|          | 1/10012 [00:00<25:36,  6.51it/s, loss=1.68, perplexity=5.38]

======================================== GPU Details ========================================
  id  name                  load    free memory    used memory    total memory    temperature    uuid
----  --------------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla P100-PCIE-16GB  43.0%   7925.0MB       8355.0MB       16280.0MB       39.0 °C        GPU-74642858-7aa9-d838-71ef-babcc3a0a23f


Train:  10%|█         | 1002/10012 [02:32<26:10,  5.74it/s, loss=2.44, perplexity=11.5]

======================================== GPU Details ========================================
  id  name                  load    free memory    used memory    total memory    temperature    uuid
----  --------------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla P100-PCIE-16GB  79.0%   3535.0MB       12745.0MB      16280.0MB       68.0 °C        GPU-74642858-7aa9-d838-71ef-babcc3a0a23f


Train:  20%|█▉        | 2001/10012 [05:02<17:49,  7.49it/s, loss=2.56, perplexity=12.9]

...

======================================== GPU Details ========================================
  id  name                  load    free memory    used memory    total memory    temperature    uuid
----  --------------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla P100-PCIE-16GB  3.0%    3529.0MB       12751.0MB      16280.0MB       54.0 °C        GPU-74642858-7aa9-d838-71ef-babcc3a0a23f


Train:  50%|████▉     | 5001/10012 [16:49<12:22,  6.74it/s, loss=2.72, perplexity=15.2]

======================================== GPU Details ========================================
  id  name                  load    free memory    used memory    total memory    temperature    uuid
----  --------------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  Tesla P100-PCIE-16GB  0.0%    883.0MB        15397.0MB      16280.0MB       54.0 °C        GPU-74642858-7aa9-d838-71ef-babcc3a0a23f


Train:  50%|█████     | 5044/10012 [18:19<11:10,  7.41it/s, loss=2.74, perplexity=15.5]

KeyboardInterrupt: ignored

In [ ]:
def choose_answer(variants):
  for beam_output in variants:
    i = tokenizer.decode(beam_output)
    # print(i)
    if i[-1] in '?.!':
      return i
  
  ans = tokenizer.decode(variants[0]).split('.')
  # print('########', ' '.join(ans[:-1]))
  return ' '.join(ans[:-1])

In [ ]:
def speak():
  
  about = '<bos> Меня Зовут Анна.'

  history = []
  print('Начните диалог')

  while(True):
    message = input()
    i = 1
    history.append([' <speaker1> ' + message])
    input_text = about + ''.join(list(chain(*history))) + ' <speaker2> <fos>' # history[-4:]

    inds = tokenizer(input_text, return_tensors='pt')['input_ids'].to(device)
    beam_outputs = model.generate(
          input_ids=inds,
          top_p=1, 
          temperature=1,
          max_length= len(inds[0]) + 20, 
          min_length= len(inds[0]) + 20, 
          num_beams=5, 
          no_repeat_ngram_size=2, 
          num_return_sequences=5, 
          # early_stopping=True,
          eos_token_id = 50259,
          pad_token_id = 0,
          )

    answer = choose_answer(beam_outputs)

    history.append([' <speaker2> ' + answer[len(input_text):]])
    # print(input_text)
    # # print(answer)
    print(answer[len(input_text):])
    # print(history)



# repetition_penalty=5.0,
#           top_k=5, 
#           top_p=0.95, 
#           temperature=1,
#           max_length= len(inds[0]) + 20,
#           do_sample=True,
#           no_repeat_ngram_size=10,
#           pad_token_id = 50259,
#           # length_penalty = 0.5

In [ ]:
speak()

Начните диалог
Привет!
<bos> Меня Зовут Анна. <speaker1> Привет! <speaker2> <fos> Как тебя зовут? Чем занимаешься? Кем работаешь?. Чем увлекаешься в свободное время?).
 Как тебя зовут? Чем занимаешься? Кем работаешь?. Чем увлекаешься в свободное время?).
Я Ваня, студент, а тебя как зовут?
<bos> Меня Зовут Анна. <speaker1> Привет! <speaker2>  Как тебя зовут? Чем занимаешься? Кем работаешь?. Чем увлекаешься в свободное время?). <speaker1> Я Ваня, студент, а тебя как зовут? <speaker2> <fos>Меня зовут Аня, я работаю в банке. А ты чем любишь заниматься? У тебя есть семья?
Меня зовут Аня, я работаю в банке. А ты чем любишь заниматься? У тебя есть семья?
да, есть девушка
<bos> Меня Зовут Анна. <speaker1> Привет! <speaker2>  Как тебя зовут? Чем занимаешься? Кем работаешь?. Чем увлекаешься в свободное время?). <speaker1> Я Ваня, студент, а тебя как зовут? <speaker2> Меня зовут Аня, я работаю в банке. А ты чем любишь заниматься? У тебя есть семья? <speaker1> да, есть девушка <speaker2> <fos> а у

In [ ]:

arr = [len(generate_example(i)[0]) for i in range(2000, 2420)]


In [ ]:
print(np.argmax(arr), max(arr))


In [ ]:
np.mean(arr)

In [ ]:
get_example(1)

In [ ]:
tokenizer.convert_tokens_to_ids('<pad>')